In [1]:
import os 
import glob
import numpy as np
import rasterio as rio
import pandas as pd
import json
import pyproj
from rasterio import plot
from rasterio.mask import raster_geometry_mask
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
import geopandas as gpd
from geocube.api.core import make_geocube
import rioxarray as rx

## Helper functions

In [2]:
# Generate binary mask from multi-band .tif file

def binary_mask(mask_fp):
    '''This function turns a multi-band raster mask into a single-band raster mask
    with unmasked pixels coded as 1s and masked pixels coded as 0s
    
    Inputs:
    mask_fp (str) : filepath to the mask .tif file
    
    Returns:
    mask_arr_3d (np array) : 3-d numpy array of 0s and 1s'''
    
    with rio.open(mask_fp) as src:
        mask_arr = src.read()
        band_ct = mask_arr.shape[0]
    
    # get unique values for binary mask band (not necessarily 0 and 1)
    # binary mask band is the last band in the image
    
    mask_band = mask_arr[(band_ct-1)]
    mask_vals = np.unique(mask_band)
    
    # make binary mask 0s and 1s
    mask_arr_binary = mask_band*(1/(mask_vals[1]))
    
    mask_arr_3d = mask_arr_binary.reshape(1,mask_arr.shape[1],mask_arr.shape[2])
    
    #checks
    #print(mask_arr_3d.shape)
    #plot.show(mask_arr_3d)
    
    return mask_arr_3d

In [3]:
# Mask function

def mask_img(img_fp, mask_fp, output_dir):
    """
    This function masks a multispectral UAS image using a binary mask file
    The mask file must have the same dimensions and CRS as the UAV image.
    
    Inputs:
    img_fp (str) : filepath to the UAV image to be masked (.tif) 
    
    mask_fp (str) : filepath to the mask file (.tif)
    
    output_dir (str) : directory to store the masked .tif image (e.g. 'kathleen/Desktop/')
    
    Returns:
    
    A masked .tif file with the same dimensions and CRS as the original UAV image. 
    All masked pixels will have a value of 0 for all bands. Unmasked pixels will retain original band values. 
    """ 
    
    mask_arr = binary_mask(mask_fp)
    
    with rio.open(img_fp) as src:
        img_arr = src.read()
        masked_img_arr = mask_arr * img_arr
        
        kwargs = src.meta
        band_ct = masked_img_arr.shape[0]
        kwargs.update(dtype=rio.float32, count=band_ct)
        
        with rio.open(output_dir+
                      'masked_'+
                      str(os.path.basename(img_fp)),
                      'w', **kwargs) as dst:
            for b in range(masked_img_arr.shape[0]):
                dst.write_band(b+1, masked_img_arr[b].astype(rio.float32))
        
        
        #checks
        #print(masked_img_arr.shape)
        #plot.show(masked_img_arr[(band_ct-1)])
    

## Mask the RGB UAS orthomosaic  

In [4]:
# Paths to RGB orthomosaic and corresponding mask file. 

ortho = rio.open('/Volumes/CAIR_LAB/UAV_Share/2022/Hemp_2022/NY 2022/RGBmaps/Hemp_22_08_10_RGBmap.tif')

mask = rio.open('/Volumes/CAIR_LAB/UAV_Share/2022/Hemp_2022/NY 2022/PlantImages/Mask_TIF/22_08_10_ortho.tif')

In [1]:
# plot.show(ortho)

In [ ]:
# plot.show(mask)

In [ ]:
# Mask the orthomosaic with the corresponding mask .tif file

output_dir = '/Users/kathleenkanaley/Desktop/' # modify to match your file structure
mask_img(ortho, 
         mask, 
         output_dir)

## Use a SHP metadata file to extract reflectance data for each experimental unit

In [5]:
# The metadata file in this example is a SHP file containing hemp plant bounding box coordinates
# The experimental unit is one hemp plant 

# Metadata file with panel geometries
shp_path = '/Volumes/CAIR_LAB/UAV_Share/2022/Hemp_2022/NY 2022/RGBmaps/Shapefile/map_22_08_10_RGB_poly.shp'

# Read the .shp file as a geodataframe
gdf = gpd.read_file(shp_path)
gdf.head()

,plant_id,genotype,c_east,c_north,geometry
0,3000,None,336143.632833,4.751144e+06,"POLYGON ((336144.656 4751142.564, 336142.656 4..."
1,3001,None,336143.714452,4.751146e+06,"POLYGON ((336144.737 4751144.631, 336142.738 4..."
2,3002,None,336143.660039,4.751148e+06,"POLYGON ((336144.683 4751147.134, 336142.683 4..."
3,3003,None,336143.619230,4.751150e+06,"POLYGON ((336144.642 4751149.379, 336142.643 4..."
4,3004,None,336143.537610,4.751153e+06,"POLYGON ((336144.560 4751151.814, 336142.561 4..."


In [6]:
# Reset index
gdf['index'] = gdf.index
gdf

,plant_id,genotype,c_east,c_north,geometry,index
0,3000,None,336143.632833,4.751144e+06,"POLYGON ((336144.656 4751142.564, 336142.656 4...",0
1,3001,None,336143.714452,4.751146e+06,"POLYGON ((336144.737 4751144.631, 336142.738 4...",1
2,3002,None,336143.660039,4.751148e+06,"POLYGON ((336144.683 4751147.134, 336142.683 4...",2
3,3003,None,336143.619230,4.751150e+06,"POLYGON ((336144.642 4751149.379, 336142.643 4...",3
4,3004,None,336143.537610,4.751153e+06,"POLYGON ((336144.560 4751151.814, 336142.561 4...",4
...,...,...,...,...,...,...
545,3545,None,336255.818423,4.751166e+06,"POLYGON ((336256.841 4751164.587, 336254.842 4...",545
546,3546,None,336255.872836,4.751168e+06,"POLYGON ((336256.896 4751166.668, 336254.896 4...",546
547,3547,None,336255.614375,4.751170e+06,"POLYGON ((336256.637 4751169.090, 336254.638 4...",547
548,3548,None,336255.655185,4.751172e+06,"POLYGON ((336256.678 4751171.307, 336254.678 4...",548


In [7]:
img_path = '/Volumes/CAIR_LAB/UAV_Share/2022/Hemp_2022/NY 2022/RGBmaps/Hemp_22_08_10_RGBmap.tif' # path to masked image
img_data = rx.open_rasterio(img_path)#, masked=True)#.rio.clip(gdf.geometry.values, gdf.crs)
img_data

<xarray.DataArray (band: 3, y: 3065, x: 10403)>
[95655585 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3
  * x            (x) float64 3.361e+05 3.361e+05 ... 3.363e+05 3.363e+05
  * y            (y) float64 4.751e+06 4.751e+06 ... 4.751e+06 4.751e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        255
    STATISTICS_MEAN:           145.96114519427
    STATISTICS_MINIMUM:        59
    STATISTICS_STDDEV:         44.100737227924
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0
    units:                     ('metre', 'metre', 'metre')

In [8]:
# # Hemp
out_grid = make_geocube(
    vector_data=gdf,
    measurements=['plant_id','genotype','index'],
    like=img_data, # ensure the data are on the same grid
)

In [9]:
out_grid

<xarray.Dataset>
Dimensions:      (y: 3065, x: 10403)
Coordinates:
  * y            (y) float64 4.751e+06 4.751e+06 ... 4.751e+06 4.751e+06
  * x            (x) float64 3.361e+05 3.361e+05 ... 3.363e+05 3.363e+05
    spatial_ref  int64 0
Data variables:
    plant_id     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    index        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [10]:
# This section is specific to RGB images

blue = img_data[0]
green = img_data[1]
red = img_data[2]

band_dict = {'blue':blue, 'green':green, 'red':red}

In [11]:
# merge the dfs together

for key, b in band_dict.items():
    out_grid[key] = (b.dims, b.values, b.attrs, b.encoding)

out_grid


<xarray.Dataset>
Dimensions:      (y: 3065, x: 10403)
Coordinates:
  * y            (y) float64 4.751e+06 4.751e+06 ... 4.751e+06 4.751e+06
  * x            (x) float64 3.361e+05 3.361e+05 ... 3.363e+05 3.363e+05
    spatial_ref  int64 0
Data variables:
    plant_id     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    index        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) uint8 255 255 255 255 255 255 ... 160 214 195 177 169
    green        (y, x) uint8 255 255 255 255 255 255 ... 255 255 255 255 255
    red          (y, x) uint8 255 255 255 255 255 255 ... 126 149 138 124 124

In [12]:
# Change 255 to NAN - hemp
out_grid_nans= out_grid.where(out_grid != 255)

In [13]:
out_grid_nans

<xarray.Dataset>
Dimensions:      (y: 3065, x: 10403)
Coordinates:
  * y            (y) float64 4.751e+06 4.751e+06 ... 4.751e+06 4.751e+06
  * x            (x) float64 3.361e+05 3.361e+05 ... 3.363e+05 3.363e+05
    spatial_ref  int64 0
Data variables:
    plant_id     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    index        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) float32 nan nan nan nan nan ... 214.0 195.0 177.0 169.0
    green        (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    red          (y, x) float32 nan nan nan nan nan ... 149.0 138.0 124.0 124.0

In [15]:
# Get a dataframe with per-pixel reflectance values - hemp
outgrid_df = out_grid_nans.to_dataframe()
outgrid_df.sort_values(by=['plant_id'], inplace=True)
outgrid_df.reset_index(inplace=True)
outgrid_df.dropna(subset=['plant_id'], inplace=True) # remove pixels not associated with a plant_id
outgrid_df

,y,x,plant_id,index,blue,green,red,spatial_ref
0,4.751143e+06,336143.857286,3000.0,0.0,109.0,133.0,131.0,0
1,4.751143e+06,336143.394777,3000.0,0.0,124.0,142.0,147.0,0
2,4.751143e+06,336143.381174,3000.0,0.0,122.0,155.0,152.0,0
3,4.751143e+06,336143.367570,3000.0,0.0,120.0,141.0,133.0,0
4,4.751143e+06,336143.353967,3000.0,0.0,98.0,118.0,117.0,0
...,...,...,...,...,...,...,...,...
11885000,4.751175e+06,336255.335510,3549.0,549.0,22.0,65.0,35.0,0
11885001,4.751175e+06,336255.321906,3549.0,549.0,22.0,55.0,35.0,0
11885002,4.751175e+06,336255.308303,3549.0,549.0,15.0,41.0,32.0,0
11885003,4.751175e+06,336255.281097,3549.0,549.0,14.0,30.0,31.0,0


## Optionally, calculate the average reflectance for each experimental unit

In [22]:
# Calculate the average reflectance for each experimental uit (in this case, experimental unit = one panel)
groupby_plantid = out_grid_nans.drop("spatial_ref").groupby(out_grid_nans.index)

plant_means = groupby_plantid.mean()
as_df = plant_means.to_dataframe()
as_df

/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_2672/1448236188.py:2: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  groupby_plantid = out_grid_nans.drop("spatial_ref").groupby(out_grid_nans.index)


,plant_id,blue,green,red
index,,,,
0.0,3000.0,118.482468,150.259155,133.993484
1.0,3001.0,149.623581,175.818176,173.514862
2.0,3002.0,130.118362,170.812912,155.690811
3.0,3003.0,145.516113,182.591217,168.443161
4.0,3004.0,119.155701,147.724915,132.744949
...,...,...,...,...
545.0,3545.0,108.302902,149.136871,139.069534
546.0,3546.0,110.758827,150.639786,150.062515
547.0,3547.0,134.798782,161.242249,154.998474


In [24]:
as_df.sort_values(by=['plant_id'], inplace=True)
as_df.reset_index(inplace=True)
as_df

,index,plant_id,blue,green,red
0,0.0,3000.0,118.482468,150.259155,133.993484
1,1.0,3001.0,149.623581,175.818176,173.514862
2,2.0,3002.0,130.118362,170.812912,155.690811
3,3.0,3003.0,145.516113,182.591217,168.443161
4,4.0,3004.0,119.155701,147.724915,132.744949
...,...,...,...,...,...
544,545.0,3545.0,108.302902,149.136871,139.069534
545,546.0,3546.0,110.758827,150.639786,150.062515
546,547.0,3547.0,134.798782,161.242249,154.998474
547,548.0,3548.0,102.397469,140.224716,128.516266


In [25]:
final_df = as_df.drop(['index'], axis=1)
final_df

,plant_id,blue,green,red
0,3000.0,118.482468,150.259155,133.993484
1,3001.0,149.623581,175.818176,173.514862
2,3002.0,130.118362,170.812912,155.690811
3,3003.0,145.516113,182.591217,168.443161
4,3004.0,119.155701,147.724915,132.744949
...,...,...,...,...
544,3545.0,108.302902,149.136871,139.069534
545,3546.0,110.758827,150.639786,150.062515
546,3547.0,134.798782,161.242249,154.998474
547,3548.0,102.397469,140.224716,128.516266


## Save the dataframe as a CSV

In [173]:
## Per-pixel
# outgrid_df.to_csv('/Users/kathleenkanaley/Desktop/perpixel_hemp_22_08_10.csv',index=False)

## Per-plant (experimental unit)
#final_df.to_csv('/Users/kathleenkanaley/Desktop/perplant_hemp_22_08_10.csv',index=False)